<a href="https://colab.research.google.com/github/ohashi-gnct/exp/blob/2022/B_%E4%BB%AE%E6%83%B3%E9%80%9A%E8%B2%A8%E3%81%A8%E6%A0%AA%E4%BE%A1%E6%8C%87%E6%95%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import io
import json
import datetime as dt

import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns

Colabに最初から入っていないライブラリをUNIXコマンドの`pip`で導入する。

`japanize-matplotlib`は日本語でmatplotlibが使えるようにするライブラリである。

In [ ]:
!pip install japanize-matplotlib

In [ ]:
# matplotlibの日本語化
import japanize_matplotlib
%matplotlib inline
sns.set(font="IPAexGothic")

Googleドライブを接続し、CSVファイルを自分のドライブに保存する。

認証を求める画面が出る場合がある。

自身でJupyter notebook環境を作っている場合、ディレクトリは適当に読み替えること。

In [ ]:
# Googleドライブの接続
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
drive_path = '/content/drive/MyDrive'

プロットの見栄えがするように初期設定する。

In [ ]:
# matplotlibの初期設定
plt_config = {
    "figure.dpi": 150,
    "font.size": 14,
    "axes.xmargin": 0,
    "axes.ymargin": 0,
    "xtick.direction": "in",
    "ytick.direction": "in",
    "mathtext.cal": "serif",
    "mathtext.rm": "serif",
    "mathtext.it": "serif:italic",
    "mathtext.bf": "serif.bold",
    "mathtext.fontset": "cm",
    "legend.framealpha": 1.0,
    "legend.edgecolor": "black",
    "legend.fancybox": False
}
plt.rcParams.update(plt_config)

# 仮想通貨

CoinGeckoのAPIでJSONを取得し、仮想通貨の価格推移を取得する。

APIの使い方

https://www.coingecko.com/ja/api/documentation

In [ ]:
!pip install pycoingecko

In [ ]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

`bitcoin`や`ripple`など、いろいろな仮想通貨の価格が取得できる。

今回はイーサリアムの価格を取得する。

イーサリアムは有名な仮想通貨のひとつであり、コロナ禍で個人でのマイニングが流行している。

1日ごとの価格をJSON形式で取得し、pandasのDataFrameに変換する。

In [ ]:
ticker = "ethereum" # 仮想通貨名

url_crypto = "https://api.coingecko.com/api/v3/coins/" + ticker + \
             "/market_chart?vs_currency=usd&days=max"
r_crypto = requests.get(url_crypto)
dict_crypto = json.loads(r_crypto.text)
df_crypto = pd.DataFrame(dict_crypto["prices"][:-1], 
                         columns=["date", "price"])
df_crypto["date"] = pd.to_datetime(df_crypto["date"], unit="ms")
df_crypto = df_crypto.set_index("date")
df_crypto

In [ ]:
df_crypto.plot()

データはCSVとして保存する。

In [ ]:
df_crypto.to_csv(os.path.join(drive_path, "crypto.csv"))

In [ ]:
# 保存したCSVファイルを読み込みたい場合、以下のように記述する。
# df_crypto = pd.read_csv(os.path.join(drive_path, ".csv"))
# df_crypto["date"] = pd.to_datetime(df_trends["date"])
# df_crypto = df_trends.set_index("date")

# 米国株式指数

yahoo-finance-api2を利用する。

このライブラリは、[Yahoo Finance](https://finance.yahoo.com/)が非公式に提供しているAPIを使いやすくしたものである。

ドキュメントは以下の通り

https://pypi.org/project/yahoo-finance-api2/

In [ ]:
!pip install yahoo_finance_api2

In [ ]:
import sys
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError

取得する指数を指定する。

指数一覧は以下で確認できる。

https://optrip.xyz/?p=4467

これに加え、`AAPL`などの個別株も取得できる。

今回はS&P500という、アメリカの代表的な500社の株価指数を取得する。

データは平日のみであることに注意する。

In [ ]:
stock_code = "^GSPC" # 指数の名前
my_share = share.Share(stock_code)

try:
  symbol_data = my_share.get_historical(share.PERIOD_TYPE_YEAR, 3, 
                                        share.FREQUENCY_TYPE_DAY, 1)
except YahooFinanceError as e:
  print(f">>> [{stock_code}] : {e.message}")

df_stock = pd.DataFrame(symbol_data)
df_stock.columns=["date", 'Open', 'High', 'Low', 'Close', 'Volume']
# タイムスタンプを変換
df_stock["date"] = pd.to_datetime(df_stock["date"], unit="ms")
# 時間や分を含むので、他のデータと合わせるためにDayに丸める
df_stock["date"] = df_stock["date"].dt.round("D")
df_stock = df_stock.set_index("date")
df_stock

データはCSVとして保存する。

In [ ]:
df_stock.to_csv(os.path.join(drive_path, "stock.csv"))

In [ ]:
# 保存したCSVファイルを読み込みたい場合、以下のように記述する。
# df_crypto = pd.read_csv(os.path.join(drive_path, ".csv"))
# df_crypto["date"] = pd.to_datetime(df_trends["date"])
# df_crypto = df_trends.set_index("date")

株価は通常**ローソク足チャート**という図で表される。

データ解析の際は、終値`Close`だけ用いればよいが、

すべての列を使ってローソク足チャートを描くこともできる。

In [ ]:
!pip install mplfinance

In [ ]:
import mplfinance

In [ ]:
# 直近50日を表示
mplfinance.plot(df_stock[-50:], type="candle")

# 相関関係の分析

上のデータから任意の2データを選び、相関関係を分析する。

まずはDataFrameを結合する。

`index`つまり`date`が共通の列だけを結合した新たなDataFrameを作る。

In [ ]:
# 2つのdfでindexが共通の列だけを結合する
df_concat = pd.concat([df_crypto, df_stock], axis=1, join="inner")
df_concat = df_concat.drop(["Open", "High", "Low", "Volume"], axis=1)
df_concat = df_concat.set_axis([ticker, stock_code], axis=1)
df_concat

この解析では過去1年間のデータを見る。

配列の**スライス**を使って過去1年間のデータを取り出す。

In [ ]:
df_concat = df_concat[-253:]
df_concat

それぞれの変数をプロットしてみる。

In [ ]:
df_concat.plot(subplots=True)

見た目としては似た傾向にある。

株価とリンクしているところはどのような要因があったか、リンクしていないところはどのような要因化考察してもよい。

発表では相関分析や回帰分析以外の考察を説明してもよい。

仮想通貨のほうが値動きが激しいことに注意する。

matplotlibのラッパーである`seaborn`ライブラリを使って、

すべての組み合わせの2変数で散布図を作成できる。

In [ ]:
sns.pairplot(df_concat)

散布図を描くことで、2変数がどのように関係しているか直感的に確認できる。

この図をもとに考察してもよい。

これらの相関係数を求める。

3変数以上ある場合にも、相互に相関係数を求められる。

In [ ]:
corr = df_concat.corr()
sns.heatmap(corr, vmax=1, vmin=-1, center=0, 
            cmap="seismic", square=True, annot=True)

やや相関があることがわかる。

# 単回帰分析

任意の1変数を目的変数(y軸)、別の任意の1変数を説明変数(x軸)として、

単回帰分析を行う。

まず、機械学習のためのライブラリscikit-learnから

単回帰分析のためのモジュールを読み込む。

In [ ]:
from sklearn import linear_model

In [ ]:
clf = linear_model.LinearRegression()
X = df_concat[[ticker]]
Y = df_concat[[stock_code]]
clf.fit(X, Y)

print("回帰係数: " + str(clf.coef_))
print("切片: " + str(clf.intercept_))
print("決定係数: " + str(clf.score(X, Y)))

決定係数は高くないため、このモデルでは目的変数を十分に説明できているわけではない。

回帰直線を散布図に重ねて描くと以下のようになる。

In [ ]:
sns.lmplot(x=ticker, y=stock_code, data=df_concat)

グラフからも、単回帰によって十分に目的変数を説明できないことがわかる。